线性回归

In [ ]:
# 内嵌画图
import matplotlib.pyplot as plt
import random
import torch
from d2l import torch as d2l
# 没有以下这段会出现 libiomp5md.dll文件冲突的 bug
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


# 生成 y = Xw + b + 噪声
def synthetic_data(w, b, num_exaples):
    X = torch.normal(0, 1, (num_exaples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

# 设置真实权重和偏置值
true_w = torch.tensor([2, -3.4])
true_b = 4.2
# 生成数据集
features, labels = synthetic_data(true_w, true_b, 1000)
# 打印形状
print("features.shape:", features.shape)

d2l.set_figsize()
d2l.plt.scatter(features[:, (1)].detach().numpy(), labels.detach().numpy(), 1)


# 遍历数据集并不断读取小批量数据样本。该函数返回 batch_size个随机样本的特征数量和标签
def data_iter(batch_size, features, labels):
    num_examples = len(features)  # 样本个数
    indices = list(range(num_examples))  # 样本索引
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            # 当i+batch_size超出时，取num_examples
            indices[i:min(i + batch_size, num_examples)])
        # 获得随即顺序的特征，及对应的标签
        yield features[batch_indices], labels[batch_indices]


batch_size = 10
for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)  # 取一个批次后，就break跳出了
    break

# 定义初始化模型参数
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


# 定义模型
def linreg(X, w, b):
    return torch.matmul(X, w) + b  # 公式为 l = Xw + b


# 定义均方损失函数
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2  # 将 y统一成与 y_hat一样同尺寸


# 定义小批量随机梯度下降的优化算法，通过更新参数来最小化损失
def sgd(params, lr, batch_size):
    with torch.no_grad():  # 不要产生梯度计算，减少内存消耗
        for param in params:  # 每个参数进行遍历
            # 每个参数进行更新，损失函数没有求均值，所以这里除以 batch_size 求了均值。
            # 由于乘法的线性关系，这里除以放在 loss的除以是等价的。
            param -= lr * param.grad / batch_size
            param.grad.zero_()  # 每个参数的梯度清零


# 训练过程
# 设置超参数，通过调整超参数降低损失函数的值
lr = 0.03  # lr = 0.03表示学习率为 0.03，即每次更新参数时的步长
num_epochs = 3  # 拟合次数
net = linreg  # 这里用线性模型，这样写是很方便net赋予其他模型，只需要改一处，不需要下面所有网络模型名称都改
loss = squared_loss

# 训练过程
for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # x和 y的小批量损失
        # 因为l是形状是(batch_size,1)，而不是一个标量。l中所有元素被加到一起
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()  # loss.backward()用于计算损失函数对于模型参数的梯度，以便进行反向传播和参数更新
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        # 每次训练打印当前周期的损失
        print(f'epoch{epoch + 1},loss{float(train_l.mean()):f}')

    # 比较真实参数和通过训练学到的参数来评估训练的成功程度

print(f'w的估计误差：{true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差：{true_b - b}')
# 打印图像
plt.show()

线性回归（使用框架）

In [ ]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
from torch import nn # neural network的缩写

true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w,true_b,1000)


# 调用框架现有的API来读取数据
def load_array(data_arrays,batch_size,is_Train=True):
    dataset = data.TensorDataset(*data_arrays)
    # 返回的是从dataset中随机挑选出batch_size个样本
    return data.DataLoader(dataset,batch_size,shuffle=is_Train)

batch_size = 10
data_iter = load_array((features,labels),batch_size)
print(next(iter(data_iter)))  # next是取迭代器里面的元素

# torch.nn.sequential 是 PyTorch 中的一个神经网络容器，用于将多个层组合在一起。
# 它可以帮助简化网络构建的过程，使得网络的结构更加清晰。
# 使用 Sequential 可以直接将各个层按顺序添加到网络中，而无需手动定义前向传播过程。
# nn.Linear做线性变换
net = nn.Sequential(nn.Linear(2,1))

net[0].weight.data.normal_(0,0.01)  # 使用正态分布替换掉 weight变量里面的数据值
net[0].bias.data.fill_(0)  # 偏差 bias变量的值设置为0
print(net[0])

# nn.MSELoss是均方损失函数
loss = nn.MSELoss()

# torch.optim.SGD是 PyTorch中的一个随机梯度下降优化器
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

num_epochs = 3

for epoch in range(num_epochs):
    for X,y in data_iter:
        # print('X = ',X)
        # print('y = ',y)
        l = loss(net(X),y)
        trainer.zero_grad()  # 梯度清零
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch+1},loss = {l:f}')